#### This Version Creates the Graph in Neo4J then runs the Queries.

In [1]:
from neo4j import GraphDatabase
import os
import pandas as pd
import shutil
import subprocess

This moves the nodes and relationships files from data/current_working_graph into the DBMS import folder.

At the moment this uses local paths, but the rest should be automated.

In [2]:
# Define the source directory
source_dir = os.path.abspath('../../data/graph_2_3')

# Define the destination directory
# This should be the import directory of your graph database
destination_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/import'

# Specify the filenames to move
filenames = ['nodes.csv', 'relationships.csv']

for filename in filenames:
    source_path = os.path.join(source_dir, filename)
    destination_path = os.path.join(destination_dir, filename)
    
    # Move each file
    shutil.copy(source_path, destination_path)

Then run the admin import command.

In [3]:
# Define the working directory
working_dir = 'C:/Users/stang/.Neo4jDesktop/relate-data/dbmss/dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351/bin'

# Construct the command
# command = [
#     './bin/neo4j-admin', 'database', 'import', 'full',
#     '--nodes=import/nodes.csv',
#     '--relationships=import/relationships.csv', 'neo4j'
# ]

command = 'neo4j-admin database import full --nodes=import/nodes.csv --relationships=import/relationships.csv --overwrite-destination neo4j'

# Execute the command


result = subprocess.run(command, shell=True, cwd=working_dir, capture_output=True, text=True)
#result = subprocess.run(command, cwd=working_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Check if the command was successful
if result.returncode == 0:
    print("Import successful")
    print(result.stdout)
else:
    print("Error during import:")
    print(result.stderr)

Import successful
Neo4j version: 5.12.0
Importing the contents of these files into C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\data\databases\neo4j:
Nodes:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\nodes.csv

Relationships:
  C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\relationships.csv


Available resources:
  Total machine memory: 31.86GiB
  Free machine memory: 19.72GiB
  Max heap memory : 910.5MiB
  Max worker threads: 8
  Configured max memory: 17.02GiB
  High parallel IO: true

Cypher type normalization is enabled (disable with --normalize-types=false):
  Property type of 'Weight' normalized from 'float' --> 'double' in C:\Users\stang\.Neo4jDesktop\relate-data\dbmss\dbms-d0a05d27-d2ec-404f-a7c3-e7f4b6a97351\import\relationships.csv

Import starting 2024-03-15 10:59:51.695+0000
  Estimated number of nodes: 3.05 k
  Estimated numbe

Run and connect to the Neo4j Database

In [7]:
from neo4j import GraphDatabase

uri = "neo4j://localhost:7687"
username = "neo4j"              # Neo4J username
#password = os.environ['NEO4J_Password']           # Neo4J password
password = '26622002'

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

# Ensure you close the driver connection when your program ends
def close_driver():
    driver.close()

**First create the gds Graph Projection**

In [8]:
def project_graph(tx):
    query = """
    CALL gds.graph.project(
      'myGraph', 
      ['Normal', 'Hyperglycemia', 'Hypoglycemia'],
      {
        LINK: {
          orientation: 'UNDIRECTED',
          properties: 'Weight'
        }
      }
    )
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(project_graph)

Define the graph algorithms.

In [9]:
def run_pagerank_centrality(tx):
    query = """
    CALL gds.pageRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [10]:
def run_degree_centrality(tx):
    query = """
    CALL gds.degree.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [11]:
def run_eigenvector_centrality(tx):
    query = """
    CALL gds.eigenvector.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [12]:
def run_articlerank_centrality(tx):
    query = """
    CALL gds.articleRank.stream('myGraph', {
        relationshipWeightProperty: 'Weight'
    })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).SpecID AS name, score
    ORDER BY score DESC, name ASC
    """
    results = tx.run(query)
    return [(record["name"], record["score"]) for record in results]

In [13]:
def run_label_propagation_algorithm(tx):
    query = """
    CALL gds.labelPropagation.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [14]:
def run_leiden_algorithm(tx):
    query = """
    CALL gds.leiden.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [15]:
def run_louvain_algorithm(tx):
    query = """
    CALL gds.louvain.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, communityId AS Community
    RETURN gds.util.asNode(nodeId).SpecID AS name, Community
    ORDER BY Community, name
    """
    results = tx.run(query)
    return [(record["name"], record["Community"]) for record in results]

In [16]:
def run_node2vec_algorithm(tx):
    query = """
    CALL gds.node2vec.stream('myGraph', { relationshipWeightProperty: 'Weight' })
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

In [17]:
def run_fastRP_algorithm(tx):
    query = """
    CALL gds.fastRP.stream('myGraph',
        { relationshipWeightProperty: 'Weight',
         randomSeed:1234,
         embeddingDimension: 128
        }
    )
    YIELD nodeId, embedding
    RETURN gds.util.asNode(nodeId).SpecID AS name, embedding
    """
    results = tx.run(query)
    return [(record["name"], record["embedding"]) for record in results]

Execute the algorithms and store the results in a Dataframe.

In [18]:
# Use a session to execute the queries and retrieve the results
with driver.session() as session:
    pagerank_results = session.execute_read(run_pagerank_centrality)
    degree_results = session.execute_read(run_degree_centrality)
    eigenvector_results = session.execute_read(run_eigenvector_centrality)
    articlerank_results = session.execute_read(run_articlerank_centrality)
    label_propagation_results = session.execute_read(run_label_propagation_algorithm)
    leiden_results = session.execute_read(run_leiden_algorithm)
    louvain_results = session.execute_read(run_louvain_algorithm)

In [19]:
pagerank_df = pd.DataFrame(pagerank_results, columns=['name', 'PageRank'])
degree_df = pd.DataFrame(degree_results, columns=['name', 'DegreeCentrality'])
eigenvector_df = pd.DataFrame(eigenvector_results, columns=['name', 'EigenvectorCentrality'])
articlerank_df = pd.DataFrame(articlerank_results, columns=['name', 'ArticleRank'])
label_propagation_df = pd.DataFrame(label_propagation_results, columns=['name', 'LabelPropagation'])
leiden_df = pd.DataFrame(leiden_results, columns=['name', 'Leiden'])
louvain_df = pd.DataFrame(louvain_results, columns=['name', 'Louvain'])

In [20]:
merged_df = pagerank_df
for df in [degree_df, eigenvector_df, articlerank_df, leiden_df, louvain_df]:
    merged_df = pd.merge(merged_df, df, on=['name'], how='left')

In [21]:
df = merged_df.rename(columns={'name' : 'SpecID'})
df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain
0,210519-1-20,1.107603,2430.902402,0.021236,0.242349,1122,710
1,210519-1-03,1.103958,2421.011345,0.021149,0.241981,219,963
2,210407-1-22,1.102085,2417.055728,0.021115,0.241821,1122,710
3,210504-1-15,1.101968,2416.626314,0.021111,0.241806,219,963
4,210511-1-11,1.097574,2406.012276,0.021018,0.241398,1122,710
...,...,...,...,...,...,...,...
3040,210211-1-00,0.486189,848.979346,0.007416,0.182298,219,1555
3041,210303-1-48,0.482353,838.240426,0.007323,0.181903,1700,963
3042,210304-2-02,0.480946,835.660586,0.007300,0.181792,219,1555
3043,210211-1-08,0.459206,777.958531,0.006796,0.179627,219,1555


In [22]:
with driver.session() as session:
    fastRP_results = session.execute_read(run_fastRP_algorithm)

fastRP_df = pd.DataFrame(fastRP_results, columns=['SpecID', 'embeddings'])

# Expand the embeddings list into separate columns
embeddings_df = pd.DataFrame(fastRP_df['embeddings'].tolist(), index=fastRP_df.index)

# Optionally, rename the new columns
embeddings_df.columns = [f'embedding_{i}' for i in range(embeddings_df.shape[1])]

# Join the new embeddings columns to the original DataFrame
fastRP_df = pd.concat([fastRP_df.drop(['embeddings'], axis=1), embeddings_df], axis=1)
fastRP_df.to_csv('../../data/fastRP_embeddings.csv', index=False)
fastRP_df.head()

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,201210-1-00,-0.072397,0.090853,0.339611,0.077927,0.305602,0.009396,-0.164280,-0.048329,0.131896,...,-0.101606,0.107743,-0.085236,0.202271,0.290692,-0.104954,0.085658,-0.258258,-0.032510,-0.109230
1,201210-1-01,-0.072531,0.090821,0.338918,0.078490,0.306012,0.009005,-0.163757,-0.048784,0.132180,...,-0.101441,0.108169,-0.084984,0.202289,0.290697,-0.104563,0.085644,-0.258862,-0.032614,-0.109462
2,201210-1-02,-0.071833,0.091303,0.339617,0.077598,0.305434,0.009887,-0.164828,-0.048764,0.131605,...,-0.101165,0.107078,-0.085159,0.201874,0.291123,-0.104974,0.085480,-0.258050,-0.032238,-0.108675
3,201210-1-03,-0.072973,0.090813,0.339502,0.077228,0.304422,0.009518,-0.164753,-0.048064,0.131976,...,-0.101196,0.107607,-0.085007,0.203306,0.290675,-0.105502,0.085889,-0.258099,-0.031907,-0.108789
4,201210-1-04,-0.072830,0.090989,0.338916,0.078248,0.305626,0.008919,-0.163872,-0.048831,0.132158,...,-0.101277,0.107923,-0.085052,0.202431,0.290940,-0.104513,0.085390,-0.258670,-0.032612,-0.109514


Delete the projection

In [23]:
def delete_projection(tx):
    query = """
    CALL gds.graph.drop('myGraph')
    """
    tx.run(query)

# Use a session to execute the graph projection
with driver.session() as session:
    session.execute_write(delete_projection)

In [24]:
close_driver()

In [25]:
status_df = pd.read_csv('../../data/graph_2_3/nodes.csv')

In [26]:
status_df

,Unnamed: 0,SpecID:ID,Status:LABEL,SurID
0,0,201210-1-00,Normal,201210-1
1,1,201210-1-01,Normal,201210-1
2,2,201210-1-02,Normal,201210-1
3,3,201210-1-03,Normal,201210-1
4,4,201210-1-04,Normal,201210-1
...,...,...,...,...
3040,3040,210526-3-45,Hyperglycemia,210526-3
3041,3041,210526-3-46,Hyperglycemia,210526-3
3042,3042,210526-3-47,Hyperglycemia,210526-3
3043,3043,210526-3-48,Hyperglycemia,210526-3


In [27]:
status_df.drop(columns=['Unnamed: 0'], inplace=True)

In [28]:
graph_df = pd.merge(df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
graph_df.drop(columns=["SpecID:ID"], inplace=True)

In [29]:
fast_graph_df = pd.merge(fastRP_df, status_df, left_on="SpecID", right_on="SpecID:ID", how="inner")
fast_graph_df.drop(columns=["SpecID:ID"], inplace=True)

In [30]:
graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [31]:
fast_graph_df.rename(columns={"Status:LABEL":"Status"}, inplace=True)

In [32]:
fast_graph_df

,SpecID,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,...,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127,Status,SurID
0,201210-1-00,-0.072397,0.090853,0.339611,0.077927,0.305602,0.009396,-0.164280,-0.048329,0.131896,...,-0.085236,0.202271,0.290692,-0.104954,0.085658,-0.258258,-0.032510,-0.109230,Normal,201210-1
1,201210-1-01,-0.072531,0.090821,0.338918,0.078490,0.306012,0.009005,-0.163757,-0.048784,0.132180,...,-0.084984,0.202289,0.290697,-0.104563,0.085644,-0.258862,-0.032614,-0.109462,Normal,201210-1
2,201210-1-02,-0.071833,0.091303,0.339617,0.077598,0.305434,0.009887,-0.164828,-0.048764,0.131605,...,-0.085159,0.201874,0.291123,-0.104974,0.085480,-0.258050,-0.032238,-0.108675,Normal,201210-1
3,201210-1-03,-0.072973,0.090813,0.339502,0.077228,0.304422,0.009518,-0.164753,-0.048064,0.131976,...,-0.085007,0.203306,0.290675,-0.105502,0.085889,-0.258099,-0.031907,-0.108789,Normal,201210-1
4,201210-1-04,-0.072830,0.090989,0.338916,0.078248,0.305626,0.008919,-0.163872,-0.048831,0.132158,...,-0.085052,0.202431,0.290940,-0.104513,0.085390,-0.258670,-0.032612,-0.109514,Normal,201210-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210526-3-45,-0.072959,0.090238,0.338410,0.077975,0.305944,0.008490,-0.164176,-0.049309,0.132130,...,-0.085406,0.201807,0.291043,-0.104457,0.084954,-0.258379,-0.032226,-0.108319,Hyperglycemia,210526-3
3041,210526-3-46,-0.072652,0.090433,0.339383,0.078422,0.305809,0.009410,-0.163624,-0.048014,0.132089,...,-0.084771,0.202816,0.290263,-0.104652,0.086279,-0.258665,-0.032565,-0.109367,Hyperglycemia,210526-3
3042,210526-3-47,-0.072975,0.090322,0.339219,0.078218,0.305886,0.009406,-0.163959,-0.048832,0.132170,...,-0.085322,0.201965,0.290966,-0.104337,0.085493,-0.258314,-0.032142,-0.108555,Hyperglycemia,210526-3
3043,210526-3-48,-0.072392,0.090489,0.339247,0.078199,0.305929,0.008953,-0.164130,-0.048310,0.131927,...,-0.085027,0.202360,0.290383,-0.104925,0.085917,-0.258365,-0.032496,-0.108917,Hyperglycemia,210526-3


In [33]:
graph_df

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,SurID
0,210519-1-20,1.107603,2430.902402,0.021236,0.242349,1122,710,Hyperglycemia,210519-1
1,210519-1-03,1.103958,2421.011345,0.021149,0.241981,219,963,Hyperglycemia,210519-1
2,210407-1-22,1.102085,2417.055728,0.021115,0.241821,1122,710,Hypoglycemia,210407-1
3,210504-1-15,1.101968,2416.626314,0.021111,0.241806,219,963,Hypoglycemia,210504-1
4,210511-1-11,1.097574,2406.012276,0.021018,0.241398,1122,710,Hyperglycemia,210511-1
...,...,...,...,...,...,...,...,...,...
3040,210211-1-00,0.486189,848.979346,0.007416,0.182298,219,1555,Hyperglycemia,210211-1
3041,210303-1-48,0.482353,838.240426,0.007323,0.181903,1700,963,Normal,210303-1
3042,210304-2-02,0.480946,835.660586,0.007300,0.181792,219,1555,Hypoglycemia,210304-2
3043,210211-1-08,0.459206,777.958531,0.006796,0.179627,219,1555,Hyperglycemia,210211-1


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

def evaluate_extra_trees(df):

    # Set the Surfaces as groups
    groups = graph_df['SurID']
    X = graph_df.drop(['Status', 'SurID', 'SpecID'], axis=1)
    y = graph_df['Status']

    # Creating the Extra Trees classifier
    et = ExtraTreesClassifier(random_state=1234)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    scores = []
    for train_index, test_index in cv.split(X, y, groups):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the Extra Trees classifier
        et.fit(X_train, y_train)
        predictions = et.predict(X_test)
        
        # Evaluate the model
        score = accuracy_score(y_test, predictions)
        scores.append(score)
    
    # Displaying the results
    print(f'{et.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

In [35]:
evaluate_extra_trees(graph_df)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3283 +/- 0.0461


In [36]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold, GroupKFold
import numpy as np

# Splitting the dataframe into features (X) and target variable (y)
X = graph_df.drop(['Status', 'SpecID', 'SurID'], axis=1)
y = graph_df['Status']

# Creating the classifiers
rf = RandomForestClassifier(random_state=1234)
et = ExtraTreesClassifier(random_state=1234)

# Combining the classifiers into a list
classifiers = [rf, et]

# Performing 10-fold cross-validation for each classifier
for clf in classifiers:
    
    # Using StratifiedKFold for classification tasks
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1234)
    
    # Getting cross-validation scores
    scores = cross_val_score(clf, X, y, cv=cv, scoring='accuracy')
    
    # Displaying the results
    print(f'{clf.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

RandomForestClassifier Cross-Validation Accuracy: 0.3682 +/- 0.0307
ExtraTreesClassifier Cross-Validation Accuracy: 0.3612 +/- 0.0300


In [37]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

In [38]:
def calculate_metrics(y_test, y_pred):

    # Calculate overall accuracy
    overall_accuracy = accuracy_score(y_test, y_pred)
    print(f"Overall Accuracy: {overall_accuracy}\n")

    # Calculate precision, recall, and F1-score for each class
    report = classification_report(y_test, y_pred)
    print("\nClassification Report:")
    print(report)

    # Show the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(conf_matrix)

In [39]:
def get_feature_importances(model, X):

    # Get feature importances
    feature_importances = model.feature_importances_

    # Creating a DataFrame to display feature importances
    feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

    # Sorting the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Show the top 10 most important features
    top_10_features = feature_importance_df.head(10)

    return top_10_features

In [40]:
calculate_metrics(y_test, rf_predictions)

Overall Accuracy: 0.36617405582922824


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.35      0.32      0.33       192
 Hypoglycemia       0.36      0.37      0.36       218
       Normal       0.38      0.41      0.40       199

     accuracy                           0.37       609
    macro avg       0.37      0.37      0.36       609
 weighted avg       0.37      0.37      0.37       609


Confusion Matrix:
[[61 73 58]
 [64 80 74]
 [48 69 82]]


In [41]:
calculate_metrics(y_test, et_predictions)

Overall Accuracy: 0.3694581280788177


Classification Report:
               precision    recall  f1-score   support

Hyperglycemia       0.35      0.31      0.33       192
 Hypoglycemia       0.39      0.38      0.38       218
       Normal       0.37      0.42      0.39       199

     accuracy                           0.37       609
    macro avg       0.37      0.37      0.37       609
 weighted avg       0.37      0.37      0.37       609


Confusion Matrix:
[[60 65 67]
 [62 82 74]
 [51 65 83]]


In [42]:
get_feature_importances(rf_model, X)

,Feature,Importance
0,PageRank,0.242765
1,DegreeCentrality,0.230733
3,ArticleRank,0.230634
2,EigenvectorCentrality,0.230235
4,Leiden,0.035406
5,Louvain,0.030227


In [43]:
get_feature_importances(et_model, X)

,Feature,Importance
0,PageRank,0.240788
3,ArticleRank,0.235372
1,DegreeCentrality,0.234854
2,EigenvectorCentrality,0.230949
4,Leiden,0.031944
5,Louvain,0.026093


## FastRp

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = fast_graph_df.drop(columns=['SpecID', 'Status', 'SurID'])

# Target variable
y = fast_graph_df['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.5993431855500821
Extra Trees Accuracy: 0.5993431855500821


In [45]:
evaluate_extra_trees(fast_graph_df)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3283 +/- 0.0461


Fast rp mixed with traditional graph metrics

In [46]:
joined_graph = pd.merge(graph_df, fast_graph_df, on=['SpecID', 'Status', 'SurID'], how='inner')

In [47]:
joined_graph

,SpecID,PageRank,DegreeCentrality,EigenvectorCentrality,ArticleRank,Leiden,Louvain,Status,SurID,embedding_0,...,embedding_118,embedding_119,embedding_120,embedding_121,embedding_122,embedding_123,embedding_124,embedding_125,embedding_126,embedding_127
0,210519-1-20,1.107603,2430.902402,0.021236,0.242349,1122,710,Hyperglycemia,210519-1,-0.072864,...,-0.101361,0.108588,-0.084982,0.202496,0.290489,-0.104643,0.085637,-0.258946,-0.032535,-0.109219
1,210519-1-03,1.103958,2421.011345,0.021149,0.241981,219,963,Hyperglycemia,210519-1,-0.072825,...,-0.101989,0.109180,-0.084854,0.202363,0.289875,-0.104931,0.085761,-0.259628,-0.032851,-0.109998
2,210407-1-22,1.102085,2417.055728,0.021115,0.241821,1122,710,Hypoglycemia,210407-1,-0.072821,...,-0.101320,0.108457,-0.085092,0.202338,0.290642,-0.104678,0.085586,-0.258791,-0.032359,-0.109101
3,210504-1-15,1.101968,2416.626314,0.021111,0.241806,219,963,Hypoglycemia,210504-1,-0.072217,...,-0.102186,0.109183,-0.084658,0.202524,0.289638,-0.104759,0.086274,-0.259856,-0.032936,-0.110304
4,210511-1-11,1.097574,2406.012276,0.021018,0.241398,1122,710,Hyperglycemia,210511-1,-0.072546,...,-0.101577,0.108984,-0.084972,0.202369,0.290351,-0.104339,0.085850,-0.259399,-0.032494,-0.109582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3040,210211-1-00,0.486189,848.979346,0.007416,0.182298,219,1555,Hyperglycemia,210211-1,-0.071333,...,-0.102527,0.108772,-0.086826,0.197874,0.290546,-0.107449,0.081746,-0.257414,-0.032968,-0.106808
3041,210303-1-48,0.482353,838.240426,0.007323,0.181903,1700,963,Normal,210303-1,-0.074264,...,-0.102589,0.107668,-0.087671,0.203410,0.292300,-0.105417,0.083322,-0.259919,-0.032160,-0.109114
3042,210304-2-02,0.480946,835.660586,0.007300,0.181792,219,1555,Hypoglycemia,210304-2,-0.071588,...,-0.102608,0.108878,-0.086829,0.197891,0.290464,-0.107583,0.081670,-0.257409,-0.033069,-0.106922
3043,210211-1-08,0.459206,777.958531,0.006796,0.179627,219,1555,Hyperglycemia,210211-1,-0.072770,...,-0.103113,0.108668,-0.088590,0.199103,0.291903,-0.107319,0.080223,-0.258327,-0.032752,-0.106699


In [48]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming graph_df is your DataFrame

# Drop the SpecID column
X = joined_graph.drop(columns=['SpecID', 'Status', 'SurID'])

# Target variable
y = joined_graph['Status']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1234)
rf_model.fit(X_train, y_train)

# Extra Trees model
et_model = ExtraTreesClassifier(n_estimators=100, random_state=1234)
et_model.fit(X_train, y_train)

# Predictions
rf_predictions = rf_model.predict(X_test)
et_predictions = et_model.predict(X_test)

# Evaluate accuracy
rf_accuracy = accuracy_score(y_test, rf_predictions)
et_accuracy = accuracy_score(y_test, et_predictions)

print("Random Forest Accuracy:", rf_accuracy)
print("Extra Trees Accuracy:", et_accuracy)

Random Forest Accuracy: 0.6042692939244664
Extra Trees Accuracy: 0.5993431855500821


In [49]:
evaluate_extra_trees(joined_graph)

ExtraTreesClassifier Cross-Validation Accuracy: 0.3283 +/- 0.0461
